In [1]:
import pandas as pd
import numpy as np
import os
import sys
import sqlite3
import datetime

from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import jupyter_black

jupyter_black.load()

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from lib.olist import get_olist_dataframe
from lib.clean import clean

from lib.graphUtils import (
    show_lines,
)

pio.renderers.default = "iframe"

working_dir = "working"
if not os.path.exists(working_dir):
    os.mkdir(working_dir)


db_file = f"{working_dir}/olist.sqlite3"

# Création du dataframe de référence

In [2]:
date_1_y = datetime.datetime.strptime("2017-09-30 23:59:59", "%Y-%m-%d %H:%M:%S")

## Filtrage de la base de données 

In [3]:
df = get_olist_dataframe(
    db_file=db_file, dir_path="../assets/olist", to_purchase_date=date_1_y
)

In [4]:
df.sort_values("last_timestamp")

,customer_unique_id,last_order_id,customer_state,customer_latitude,customer_longitude,nb_all_purchases,average_amount,last_timestamp,last_nb_items,last_categories,...,last_review_score,all_purchases_timestamps,all_purchases_categories,all_purchases_amount,all_purchases_payments,all_purchases_review_score,all_expected_delivery_delay,all_effective_delivery_delay,all_freight_value,all_photos_quantity
18878,b7d76e111c89f7ebf14761390f0f7d17,2e7a8482f6fb09756ca50c10d7bfc047,RR,2.813808,-60.701637,1,72.89,2016-09-04 21:15:19,2,"furniture_decor:39.99,furniture_decor:32.9",...,1.0,2016-09-04 21:15:19,"furniture_decor:39.99,furniture_decor:32.9",72.89,credit_card:136.23,1.000000,45.114363,NaN,63.34,5.0
13534,830d5b7aaa3b6f1e9ad63703bec97d23,bfbd0f9bdef84302105ad712db648a6c,SP,-20.585829,-47.863559,1,134.97,2016-09-15 12:16:38,3,"health_beauty:44.99,health_beauty:44.99,health...",...,1.0,2016-09-15 12:16:38,"health_beauty:44.99,health_beauty:44.99,health...",134.97,None,1.000000,18.488449,54.813194,8.49,3.0
5273,32ea3bdedab835c3aa6cb68ce66565ef,3b697a20d9e427646d92567910af6d57,SP,-23.581469,-46.635030,2,63.95,2016-10-03 09:44:50,1,watches_gifts:29.9,...,4.0,"2017-09-27 08:20:09,2016-10-03 09:44:50","stationery:25.99,watches_gifts:29.9,watches_gi...",104.89,"credit_card:34.92,boleto:45.46,credit_card:6.7...",4.666667,19.623270,15.299138,15.56,3.5
4912,2f64e403852e6893ae37485d5fcacdaf,be5bc2f0da14d8071e2d45451ad119d9,RS,-28.294429,-53.502746,1,21.90,2016-10-03 16:56:50,1,sports_leisure:21.9,...,4.0,2016-10-03 16:56:50,sports_leisure:21.9,21.90,boleto:39.09,4.000000,34.293866,24.057500,17.19,1.0
10188,61db744d2f835035a5625b59350c6b63,a41c8759fbe7aab36ea07e038b2d4465,RS,-30.041351,-51.213096,1,36.49,2016-10-03 21:13:36,1,sports_leisure:36.49,...,3.0,2016-10-03 21:13:36,sports_leisure:36.49,36.49,boleto:53.73,3.000000,56.115556,30.572581,17.24,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,19090a836e2392cd957ed43d61804ba3,90f3963bbaa0e66ce36c4f9be2ed2914,SP,-23.549429,-46.545728,1,39.00,2017-09-30 22:52:23,1,sports_leisure:39.0,...,5.0,2017-09-30 22:52:23,sports_leisure:39.0,39.00,credit_card:50.73,5.000000,17.046956,2.722234,11.73,1.0
24841,f1e036dfac59643f7bdd306447f92402,349af77e71491a1aed28d6bf0528e09b,PA,-1.460540,-48.483061,1,109.90,2017-09-30 22:59:50,1,sports_leisure:109.9,...,5.0,2017-09-30 22:59:50,sports_leisure:109.9,109.90,boleto:135.7,5.000000,39.041782,22.796782,25.80,8.0
5550,3581031fd294025f4c16cd754bc4c90e,ed5495fbd35104360dbf954d7fece421,RJ,-22.886669,-43.279084,1,69.90,2017-09-30 23:05:06,1,baby:69.9,...,5.0,2017-09-30 23:05:06,baby:69.9,69.90,credit_card:89.63,5.000000,26.038125,17.972407,19.73,3.0
4622,2cb6e1938aa20ae2f3416b4c3f9f7847,b5c5a6fb605260058c01713e1fb97aba,SP,-23.556070,-46.653327,1,59.90,2017-09-30 23:17:17,1,garden_tools:59.9,...,5.0,2017-09-30 23:17:17,garden_tools:59.9,59.90,credit_card:73.34,5.000000,23.029664,8.750602,13.44,2.0


## Nettoyage du dataframe

In [5]:
r = clean(
    df,
    use_features=[
        "review",
        "weekday",
        "categories",
        "delivery",
        "customer_location",
        "payment_types",
        "photos_quantity_customer",
        "freight_value",
        "customer_unique_id",
    ],
)
df = r["data"]
scaler = r["scaler"]
pipe = r["pipe"]

featureSplitterTransformer, from_column=last_categories
featureSplitterTransformer, from_column=all_purchases_categories
featureSplitterTransformer, from_column=last_payments
featureSplitterTransformer, from_column=all_purchases_payments
last_categories
all_purchases_categories
last_payments
all_purchases_payments


In [6]:
df

,customer_latitude,customer_longitude,f,m,all_purchases_review_score,all_freight_value,all_cats_home_office,all_cats_clothes,all_cats_health_beauty,all_cats_electronics,...,all_cats_food_drink,all_cats_construction,all_payments_credit_card,all_payments_cash,all_payments_voucher,all_payments_debit_card,most_purchased_weekday,all_delivery_delay,r,customer_unique_id
0,0.131649,0.451567,0.0,0.004964,0.500,0.009074,0.010245,0.000000,0.000000,0.000000,...,0.0,0.0,0.006310,0.000000,0.0,0.0,0.666667,0.429922,0.521739,0000f46a3911fa3c0805444483337064
1,0.241766,0.575165,0.0,0.009876,0.750,0.007967,0.000000,0.039029,0.000000,0.000000,...,0.0,0.0,0.010986,0.000000,0.0,0.0,0.833333,0.348136,0.537084,0005e1862207bf6ccc02e4228effd9a0
2,0.270467,0.610770,0.0,0.000864,0.500,0.007957,0.002064,0.000000,0.000000,0.000000,...,0.0,0.0,0.002122,0.000000,0.0,0.0,0.166667,0.399298,0.189258,0006fdc98a402fceb4eb0ee528f6a8d4
3,0.078575,0.395345,0.0,0.005559,0.750,0.007530,0.000000,0.000000,0.032082,0.000000,...,0.0,0.0,0.006680,0.000000,0.0,0.0,0.666667,0.390245,0.127877,000a5ad9c4601d2bbdd9ed765d5213b3
4,0.217710,0.495528,0.0,0.002434,0.875,0.006244,0.000000,0.010119,0.000000,0.000000,...,0.0,0.0,0.003429,0.000000,0.0,0.0,0.666667,0.395550,0.002558,000bfa1d2f1a41876493be685390d6d3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26296,0.383170,0.468312,0.0,0.006148,1.000,0.009401,0.000000,0.024545,0.000000,0.000000,...,0.0,0.0,0.007519,0.000000,0.0,0.0,0.666667,0.414725,0.127877,fff3a9369e4b7102fab406a334a678c3
26297,0.217652,0.495443,0.0,0.002799,0.750,0.007957,0.000000,0.000000,0.000000,0.002969,...,0.0,0.0,0.000000,0.008176,0.0,0.0,0.666667,0.415157,0.074169,fff699c184bcc967d62fa2c6171765f7
26298,0.542440,0.715874,0.0,0.116665,1.000,0.262109,0.000000,0.000000,0.654221,0.000000,...,0.0,0.0,0.151303,0.000000,0.0,0.0,0.500000,0.351357,0.291560,fffcf5a5ff07b0908bd4e2dbc735a684
26299,0.468078,0.301095,0.0,0.006520,1.000,0.011888,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.008230,0.000000,0.0,0.0,0.166667,0.342450,0.601023,ffff371b4d645b6ecea244b27531430a


In [7]:
num_cols = df.select_dtypes(include=[np.number]).columns

In [8]:
num_cols

Index(['customer_latitude', 'customer_longitude', 'f', 'm',
       'all_purchases_review_score', 'all_freight_value',
       'all_cats_home_office', 'all_cats_clothes', 'all_cats_health_beauty',
       'all_cats_electronics', 'all_cats_culture_leisure',
       'all_cats_cool_stuff', 'all_cats_other', 'all_cats_food_drink',
       'all_cats_construction', 'all_payments_credit_card',
       'all_payments_cash', 'all_payments_voucher', 'all_payments_debit_card',
       'most_purchased_weekday', 'all_delivery_delay', 'r'],
      dtype='object')

## Décomposition PCA

In [9]:
n_components = 6
pca = PCA(n_components=n_components)

pca.fit(df.loc[:, num_cols])
pca.explained_variance_ratio_

array([0.39162227, 0.35537652, 0.13398936, 0.06753305, 0.0217489 ,
       0.00535521])

In [10]:
X_pca = pd.DataFrame(
    pca.transform(df.loc[:, num_cols]),
    columns=["pca1", "pca2", "pca3", "pca4", "pca5", "pca6"],
)

In [11]:
X_pca

,pca1,pca2,pca3,pca4,pca5,pca6
0,0.199705,0.309938,0.220999,-0.156771,-0.015100,-0.011577
1,0.374902,0.068771,0.250523,0.003052,-0.074466,-0.005350
2,-0.299111,0.284474,-0.108794,0.042658,-0.091614,-0.010138
3,0.205564,0.038887,-0.157596,-0.223125,0.014421,-0.000620
4,0.210012,-0.087752,-0.277746,-0.051665,-0.011937,-0.009443
...,...,...,...,...,...,...
26296,0.214525,-0.203323,-0.147325,0.086579,0.084895,-0.010060
26297,0.206542,0.040729,-0.213208,-0.054861,-0.012382,-0.010852
26298,0.051149,-0.193313,0.019275,0.357363,-0.058431,0.204976
26299,-0.284210,-0.195653,0.331351,0.092626,0.269468,-0.011206


## Clustering par KMeans

In [12]:
km = KMeans(init="k-means++", random_state=0, n_init="auto", n_clusters=5)
km.fit(X_pca)

KMeans(n_clusters=5, n_init='auto', random_state=0)

In [13]:
set(km.labels_)

{0, 1, 2, 3, 4}

In [14]:
df_y_1 = pd.DataFrame(
    {"customer_unique_id": df["customer_unique_id"], "cluster": km.labels_}
)

In [15]:
df_y_1

,customer_unique_id,cluster
0,0000f46a3911fa3c0805444483337064,2
1,0005e1862207bf6ccc02e4228effd9a0,0
2,0006fdc98a402fceb4eb0ee528f6a8d4,3
3,000a5ad9c4601d2bbdd9ed765d5213b3,0
4,000bfa1d2f1a41876493be685390d6d3,0
...,...,...
26296,fff3a9369e4b7102fab406a334a678c3,0
26297,fff699c184bcc967d62fa2c6171765f7,0
26298,fffcf5a5ff07b0908bd4e2dbc735a684,1
26299,ffff371b4d645b6ecea244b27531430a,4


In [16]:
week_ari_scores = []
week_ari_scores.append(1)

# Comparaison itérative

In [17]:
for w in range(1, 30):
    date_m2 = date_1_y + datetime.timedelta(days=7 * w)
    df_m2 = get_olist_dataframe(
        db_file=db_file,
        dir_path="../assets/olist",
        to_purchase_date=date_m2,
    )
    r = clean(
        df_m2,
        use_features=[
            "review",
            "weekday",
            "categories",
            "delivery",
            "customer_location",
            "payment_types",
            "photos_quantity_customer",
            "freight_value",
            "customer_unique_id",
        ],
    )
    df_m2 = r["data"]

    customers_unique_ids_y_1_m2 = df_m2.loc[
        df_m2["customer_unique_id"].isin(df_y_1["customer_unique_id"])
    ]["customer_unique_id"]

    pca_m2 = PCA(n_components=n_components)
    pca_m2.fit(df_m2.loc[:, num_cols])
    pca_m2.explained_variance_ratio_

    X_pca_m2 = pd.DataFrame(
        pca_m2.transform(df_m2.loc[:, num_cols]),
        columns=["pca1", "pca2", "pca3", "pca4", "pca5", "pca6"],
    )
    df_m2 = pd.DataFrame(
        {
            "customer_unique_id": df_m2["customer_unique_id"],
            "cluster": km.predict(X_pca_m2),
        }
    )

    ari_score = adjusted_rand_score(
        df_m2.loc[
            df_m2["customer_unique_id"].isin(customers_unique_ids_y_1_m2)
        ].sort_values("customer_unique_id")["cluster"],
        df_y_1.loc[
            df_y_1["customer_unique_id"].isin(customers_unique_ids_y_1_m2)
        ].sort_values("customer_unique_id")["cluster"],
    )
    print(f"w={w}, ari score={ari_score}")
    week_ari_scores.append(ari_score)

featureSplitterTransformer, from_column=last_categories
featureSplitterTransformer, from_column=all_purchases_categories
featureSplitterTransformer, from_column=last_payments
featureSplitterTransformer, from_column=all_purchases_payments
last_categories
all_purchases_categories
last_payments
all_purchases_payments
w=1, ari score=0.9990100317659075
featureSplitterTransformer, from_column=last_categories
featureSplitterTransformer, from_column=all_purchases_categories
featureSplitterTransformer, from_column=last_payments
featureSplitterTransformer, from_column=all_purchases_payments
last_categories
all_purchases_categories
last_payments
all_purchases_payments
w=2, ari score=0.9987862258883178
featureSplitterTransformer, from_column=last_categories
featureSplitterTransformer, from_column=all_purchases_categories
featureSplitterTransformer, from_column=last_payments
featureSplitterTransformer, from_column=all_purchases_payments
last_categories
all_purchases_categories
last_payments
all_pur

# Conclusion

In [21]:
show_lines(
    list(range(0, len(week_ari_scores))),
    {"ari": week_ari_scores},
    title="Evolution score ARI par semaine",
    h=700,
)

Le modèles est valable 7 semaines